In [1]:
import os
import time
import pandas as pd
import numpy as np

import geopandas as gpd
import rioxarray as rioxr
import rasterio

#from skimage.filters.rank import entropy
import sample_rasters as sr
from rasterio.crs import CRS

from shapely.geometry import box

import planetary_computer as pc

In [2]:
# ***************************************************
# ************* NOTEBOOK VARIABLES ******************

itemids = pd.read_csv(os.path.join(os.getcwd(),'temp', 'aoi_naip_itemids.csv'))

csv_name = 'test_set.csv'
all_pts = pd.read_csv(os.path.join(os.getcwd(), csv_name))

# ***************************************************
# ***************************************************

In [3]:
# temporary folder for aux rasters
folp = os.path.join(os.getcwd(),'temp','aux_naip_rasters')
if os.path.exists(folp) == False:
    os.mkdir(folp)

In [4]:
t0 = time.time()

sampled_pts = []

for i in [0]:
#for i in range(len(itemids)):
    # ---------------------------------------
    # open raster reader for NAIP scene
    itemid = itemids.itemid[i]
    item = sr.get_item_from_id(itemid)    
    href = pc.sign(item.assets["image"].href)
    naip_rast_r = rioxr.open_rasterio(href) 

    # ---------------------------------------
    # find polygons for that NAIP scene
    poly_fp = sr.path_to_polygons(itemids.iloc[i].aoi_name, itemids.iloc[i].year)
    polys = gpd.read_file(poly_fp)
    
    # iterate through polygons in scene
    for j in [0]:
#    for j in list(polys.id):
        # ---------------------------------------
        # find points in current polygon
        pts_poly = all_pts.loc[ (all_pts['naip_id'] == itemid) & (all_pts['polygon_id'] == j)]
        if len(pts_poly) !=0:
            crs = CRS.from_string(pts_poly.pts_crs.iloc[0])
            pts_poly_df = sr.geodataframe_from_csv(df = pts_poly, lon_label='x', lat_label='y', crs=crs)
            pts_col = pts_poly_df.to_crs(naip_rast_r.rio.crs).geometry

            # ---------------------------------------
            # create enlarged bounding box

            # (this is ugly, but unfortunately the index and the polygon.id do not match in some files)
            poly_index = polys.index[polys['id']==j].tolist()[0]                

            poly = polys.geometry[poly_index]
            reduce = gpd.GeoDataFrame({'geometry':[box(*poly.bounds)]}, crs=polys.crs)
            reduce = reduce.to_crs(naip_rast_r.rio.crs) 
            poly = reduce.geometry[0]  # poly in scene's crs
            reduce_box = box(*(poly.buffer(6).bounds)) # n must be 2*(entropy disk radius)

            # ---------------------------------------
            # clip NAIP scene
            rast = naip_rast_r.rio.clip_box(*reduce_box.bounds)

            # ---------------------------------------
            # save auxiliary entropy rasters for R,G,B,NIR bands of clipped scene
            band_names = ['r_', 'g_', 'b_', 'nir_']
            tags = ['_maxs', '_mins', '_avgs', '_entrs']
            window_fps = []
            window_cols = []

            for band_name, band_n in zip(band_names,range(1,5)):
                rast_name = band_name + itemid + '_poly_'+str(j)
                sr.max_raster(raster = rast, band=band_n, rast_name=rast_name, n=3, folder_path=folp)
                sr.min_raster(raster = rast, band=band_n, rast_name=rast_name, n=3, folder_path=folp)
                sr.avg_raster(raster = rast, band=band_n, rast_name=rast_name, n=3, folder_path=folp)
                sr.entropy_raster(raster = rast, band=band_n, rast_name=rast_name, n=3, folder_path=folp)

                for tag in tags:
                    window_fps.append(os.path.join(folp, rast_name + tag + '.tif'))        
                    window_cols.append( band_name.replace('_','')+tag.replace('s',''))

            # ------------------------------
            # make auxiliary NDVI of clipped scene
            red_band = rast.sel(band=1).astype('int16') 
            nir_band = rast.sel(band=4).astype('int16')
            ndvi = ((nir_band - red_band) / (nir_band + red_band)*100)+100

            # make auxiliary NDVI entropy
            band_names.append('ndvi_')
            rast_name = 'ndvi_' + itemid + '_poly_'+str(j)
            
            sr.max_raster(rast_data=ndvi, crs=rast.rio.crs, transf=rast.rio.transform(), rast_name=rast_name, n=3, folder_path=folp)
            sr.min_raster(rast_data=ndvi, crs=rast.rio.crs, transf=rast.rio.transform(), rast_name=rast_name, n=3, folder_path=folp)
            sr.avg_raster(rast_data=ndvi, crs=rast.rio.crs, transf=rast.rio.transform(), rast_name=rast_name, n=3, folder_path=folp)

            ndvi = ndvi*100 +100
            sr.entropy_raster(rast_data=ndvi.astype('uint8'), 
                              crs=rast.rio.crs, transf=rast.rio.transform(), 
                              rast_name=rast_name, n=3, 
                              folder_path=folp)

            for tag in tags:
                window_fps.append(os.path.join(folp, rast_name + tag + '.tif'))        
                window_cols.append( 'ndvi'+tag.replace('s',''))

            # ---------------------------------------
            # sample raster values for points in current polygon
            samples = []
            for fp, col_name in zip(window_fps, window_cols):
                rast_r = rasterio.open(fp)
                sample = sr.sample_raster_from_pts(pts_col, rast_r, [col_name])    
                samples.append(sample)

            # ---------------------------------------
            # Add all derived spectral data to pts dataframe
            new_features = pd.concat(samples, axis = 1)
            pts = pd.concat([pts_poly, new_features.set_index(pts_poly_df.index)], axis=1)                

            # -----------------------------
            # collect all points from each polygon in the scene
            sampled_pts.append(pts)

            # ---------------------------------------
            # delete aux entropy rasters
            for fp in window_fps:
                os.remove(fp)
            
sampled_pts= pd.concat(sampled_pts).sort_index()
        
print(time.time()-t0)

2.0929617881774902


In [5]:
sampled_pts.columns

Index(['x', 'y', 'pts_crs', 'aoi', 'naip_id', 'polygon_id', 'r', 'g', 'b',
       'nir', 'ndvi', 'year', 'month', 'day_in_year', 'lidar', 'max_lidar',
       'min_lidar', 'min_max_diff', 'avg_lidar', 'iceplant', 'geometry',
       'r_max', 'r_min', 'r_avg', 'r_entr', 'g_max', 'g_min', 'g_avg',
       'g_entr', 'b_max', 'b_min', 'b_avg', 'b_entr', 'nir_max', 'nir_min',
       'nir_avg', 'nir_entr', 'ndvi_max', 'ndvi_min', 'ndvi_avg', 'ndvi_entr'],
      dtype='object')

In [10]:
csv_name = 'spectral_window_test_set.csv'
win_pts =  pd.read_csv(os.path.join(os.getcwd(), csv_name))

sub = win_pts.loc[ (win_pts['naip_id'] == itemid) & (win_pts['polygon_id'] == j)]

In [8]:
sampled_pts.r_max

167000    150
167119    142
167125    144
167127    144
167144    150
         ... 
198039    140
198056    144
198162    141
198182    143
198357    145
Name: r_max, Length: 568, dtype: uint8

In [33]:
for col in window_cols:
    print(list(sub[col]) == list(sampled_pts[col]))

True
True
False
True
True
False
True
True
False
True
True
False


In [48]:
new = sampled_pts.r_avg.reset_index(drop = True)

In [49]:
old = sub.r_avg.reset_index(drop = True)

In [55]:
np.unique(new - old < 0.0001)

array([ True])